# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
# Import libraries
import pandas as pd

# Load dataset
df = pd.read_csv("homepage_actions.csv")

# Quick look at the data
print("Shape of dataset:", df.shape)
print("\nFirst few rows:")
display(df.head())

# Check unique actions
print("\nUnique actions:", df['action'].unique())

# 1. How many viewers also clicked?
# Pivot table to see counts of each action by user
user_actions = df.pivot_table(
    index="id",
    columns="action",
    aggfunc="size",
    fill_value=0
)

# Add column to flag viewers who also clicked
user_actions["view_and_click"] = (user_actions["view"] > 0) & (user_actions["click"] > 0)
print("\nNumber of users who viewed and clicked:", user_actions["view_and_click"].sum())

# 2. Did anyone click who didn’t view?
click_without_view = user_actions[(user_actions["click"] > 0) & (user_actions["view"] == 0)]
print("\nUsers who clicked without viewing:", len(click_without_view))

# 3. Check overlap between control and experiment groups
# Each user should only belong to one group ideally
groups_per_user = df.groupby("id")["group"].nunique()
overlap_users = groups_per_user[groups_per_user > 1]
print("\nNumber of users in BOTH groups (control & experiment):", overlap_users.shape[0])

# 4. Plan for overlap handling
print("\nSuggestion: Exclude users who appear in both groups from the analysis, "
      "since their presence could bias results.")


Shape of dataset: (8188, 4)

First few rows:


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view



Unique actions: ['view' 'click']

Number of users who viewed and clicked: 1860

Users who clicked without viewing: 0

Number of users in BOTH groups (control & experiment): 0

Suggestion: Exclude users who appear in both groups from the analysis, since their presence could bias results.


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
#Your code here
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest

# Keep only view and click actions
df_filtered = df[df["action"].isin(["view", "click"])]

# Create pivot by user and group
user_group = df_filtered.pivot_table(
    index=["id", "group"],
    columns="action",
    aggfunc="size",
    fill_value=0
).reset_index()

# A user is considered a "click" if they clicked at least once
user_group["clicked"] = (user_group["click"] > 0).astype(int)

# Separate control and experiment
control = user_group[user_group["group"] == "control"]
experiment = user_group[user_group["group"] == "experiment"]

# Number of clicks in each group
successes = [control["clicked"].sum(), experiment["clicked"].sum()]

# Number of total users in each group
nobs = [len(control), len(experiment)]

print("Control group CTR:", successes[0] / nobs[0])
print("Experiment group CTR:", successes[1] / nobs[1])

# Two-proportion z-test
stat, pval = proportions_ztest(successes, nobs)
print("\nZ-statistic:", stat)
print("P-value:", pval)

# Interpretation
alpha = 0.05
if pval < alpha:
    print("✅ Reject the null hypothesis: The experimental homepage is significantly more effective.")
else:
    print("❌ Fail to reject the null hypothesis: No significant difference between control and experiment.")


Control group CTR: 0.2797118847539016
Experiment group CTR: 0.3097463284379172

Z-statistic: -2.618563885349469
P-value: 0.008830075576595804
✅ Reject the null hypothesis: The experimental homepage is significantly more effective.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [3]:
#Your code here
import numpy as np
from scipy.stats import norm

# Control CTR
p_control = successes[0] / nobs[0]

# Expected clicks in experiment if CTR same as control
expected_clicks_exp = nobs[1] * p_control
expected_clicks_exp


838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [4]:
#Your code here
# Observed clicks in experiment
observed_clicks_exp = successes[1]

# Standard deviation for binomial distribution
std_dev = np.sqrt(nobs[1] * p_control * (1 - p_control))

# Z-score
z_score = (observed_clicks_exp - expected_clicks_exp) / std_dev
z_score


3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [5]:
#Your code here
# Two-tailed test p-value
p_value = 2 * (1 - norm.cdf(abs(z_score)))
p_value


0.0002497305601389943

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

Yes. The analysis code is below


In [6]:
alpha = 0.05
print("Z-score:", z_score)
print("P-value:", p_value)

if p_value < alpha:
    print("✅ Reject null: Experiment homepage is significantly different from control.")
else:
    print("❌ Fail to reject null: No significant difference.")


Z-score: 3.6625360854823588
P-value: 0.0002497305601389943
✅ Reject null: Experiment homepage is significantly different from control.


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.